In [ ]:
#| default_exp trainer

# Trainer
> Trainer API

In [ ]:
#| export
import os
import os.path as osp
from argparse import ArgumentParser
import mmcv
import torch
from pytorch_lightning import Trainer, seed_everything
from lit_classifier.all import get_trainer
import shutil
from fastcore.script import *


def get_exp_by_file(exp_file):
    """
        Params:
        exp_file: Path to exp
        
    """
    try:
        import importlib
        import os
        import sys
        sys.path.append(os.path.dirname(exp_file))
        # import ipdb; ipdb.set_trace()
        current_exp = importlib.import_module(
            os.path.basename(exp_file).split(".")[0])
        current_exp = importlib.reload(current_exp)
        exp = current_exp.Exp()
        return exp
    except Exception:
        raise ImportError(
            "{} doesn't contains class named 'Exp'".format(exp_file))
        
@call_parse
def train(
    cfg_path:Param('Path to config'),
    devices: Param('GPUS indices', default=1, type=int),
):
    cfg = get_exp_by_file(cfg_path)
    print(cfg)
    exp_name = osp.basename(cfg_path).split('.')[0]

    data = cfg.get_data_loader()

    model = cfg.get_model(create_lr_scheduler_fn=cfg.get_lr_scheduler(), 
            create_optimizer_fn=cfg.get_optimizer())
    trainer = get_trainer(exp_name,
                          devices,
                          distributed=devices > 1,
                          max_epochs=cfg.max_epochs)

    mmcv.mkdir_or_exist(trainer.log_dir)
    shutil.copy(cfg_path, osp.join(trainer.log_dir, osp.basename(cfg_path)))
    trainer.fit(model, data)


## Test

In [ ]:
#| hide
# train('./configs/00_mnist_vanila.py', devices=1)

In [ ]:
#| hide
from nbdev import nbdev_export
nbdev_export()
!pip install -e ../

Obtaining file:///home/anhvth/gitprojects/litclassifier
  Attempting uninstall: lit-classifier
    Found existing installation: lit-classifier 0.0.2
    Uninstalling lit-classifier-0.0.2:
      Successfully uninstalled lit-classifier-0.0.2
  Running setup.py develop for lit-classifier
